In [3]:
import numpy as np
import tensorflow as tf
from collections import Counter
import tensorflow_probability as tfp

In [82]:
import tensorflow as tf

def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calculates intersection over union
    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct labels of Bounding Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)
    Returns:
        tensor: Intersection over union for all examples
    """

    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    if box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]  # (N, 1)
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = tf.maximum(box1_x1, box2_x1)
    y1 = tf.maximum(box1_y1, box2_y1)
    x2 = tf.minimum(box1_x2, box2_x2)
    y2 = tf.minimum(box1_y2, box2_y2)

    # tf.maximum and tf.minimum handle the case when the boxes do not intersect
    intersection = tf.maximum(0.0, x2 - x1) * tf.maximum(0.0, y2 - y1)

    box1_area = tf.abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = tf.abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

def nms(
    bboxes,
    iou_threshold,
    prob_threshold,
    box_format = 'corners'
) :
    #bboxes = [[class, prob of bbox, x1, y1, x2, y2]]
    
    assert type(bboxes) == list
    bboxes = [box for box in bboxes if box[1] > prob_threshold]
    bboxes = sorted(bboxes, key = lambda x : x[1], reverse = True)
    bboxes_after_nms = []
    
    while bboxes :
        chosen_box = bboxes.pop(0)
        bboxes = [
            box 
            for box in bboxes
            if box[0] != chosen_box[0]
            or intersection_over_union(
                tf.convert_to_tensor(chosen_box[2:]), tf.convert_to_tensor(box[2:]), box_format=box_format
            ) < iou_threshold
        ]
        
        bboxes_after_nms.append(chosen_box)
    return bboxes_after_nms

def mean_average_precision(
    pred_boxes,
    true_boxes,
    iou_threshold = 0.5,
    box_format = 'corners',
    num_classes = 20
) : 
    #pred_boxes (list) : [[train_idx, class_pred, prob_score, x1, y1, x2, y2], ...]
    average_precisions = []
    epsilon = 1e-6
    
    for c in range(num_classes) :
        detections = []
        ground_truths = []
        
        for detection in pred_boxes :
            if detection[1] == c :
                detections.append(detection)
                
        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)
                
        amount_bboxes = Counter([gt[0] for gt in ground_truths])
        
        for key, val in amount_bboxes.items() :
            amount_bboxes[key] = tf.zeros(val)
            
            
        detections.sort(key = lambda x: x[2], reverse = True)
        TP = tf.zeros((len(detections)))
        FP = tf.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        
        for detection_idx, detection in enumerate(detections) :
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]
            num_gts = len(ground_truth_img)
            best_iou = 0
            
            for idx, gt in enumerate(ground_truth_img) :
                iou = intersection_over_union(
                    tf.convert_to_tensor(detection[3:]),
                    tf.convert_to_tensor(gt[3:]),
                    box_format=box_format
                )
                
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx
            
            if best_iou > iou_threshold:
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    temp = TP.numpy()
                    temp[detection_idx] = 1
                    TP = tf.convert_to_tensor(temp)
                    temp = amount_bboxes[detection[0]].numpy()
                    temp[best_gt_idx] = 1
                    amount_bboxes[detection[0]] = tf.convert_to_tensor(temp)
                else:
                    temp = FP.numpy()
                    temp[detection_idx] = 1
                    FP = tf.convert_to_tensor(temp)
                
            else:
                temp = FP.numpy()
                temp[detection_idx] = 1
                FP = tf.convert_to_tensor(temp)
                
                
        TP_cumsum = tf.cumsum(TP, axis = 0)
        FP_cumsum = tf.cumsum(FP, axis = 0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)          
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = tf.concat([tf.constant([1.0]), precisions], axis = 0)
        recalls = tf.concat([tf.constant([0.0]), recalls], axis = 0)
        average_precisions.append(np.trapz(precisions.numpy(), recalls.numpy()))
    
    return sum(average_precisions)  / len(average_precisions)

In [84]:
if __name__ == '__main__' :
    print('nms = ', nms([
        [1, 1, 0.5, 0.45, 0.4, 0.5],
        [1, 0.8, 0.5, 0.5, 0.2, 0.4],
        [1, 0.7, 0.25, 0.35, 0.3, 0.1],
        [1, 0.05, 0.1, 0.1, 0.1, 0.1],
    ], 
        0.2, 
        7/20, 
        box_format="midpoint",
    ))

    print('mAP = ', mean_average_precision(
        [
            [0, 0, 0.9, 0.15, 0.25, 0.1, 0.1],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ],
        [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ],
        iou_threshold=0.5,
        box_format="midpoint",
        num_classes=1,
    ))

nms =  [[1, 1, 0.5, 0.45, 0.4, 0.5], [1, 0.7, 0.25, 0.35, 0.3, 0.1]]
mAP =  0.2777775824069977
